In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

This is a (so far) failed attempt at getting langchain streaming working in gradio. Will revisit later...<br>
Reference: https://python.langchain.com/en/latest/modules/models/llms/examples/streaming_llm.html?highlight=Streaming#how-to-stream-llm-and-chat-model-responses

# langchain_gradio_streaming

> A basic gradio chat bot using OpenAI API's, langchain and gradio

In [ ]:
#| default_exp langchain_gradio_streaming

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import jarvis.secrets

In [ ]:
#| exporti
# import openai
# import numpy as np
import os
import json
import gradio as gr
# import datetime

In [ ]:
# from langchain.cache import SQLiteCache
# import langchain

In [ ]:
# langchain.llm_cache = SQLiteCache(database_path="../.langchain.db")

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.base import CallbackManager, BaseCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage

In [ ]:
#Temporary? For tracking callback
from langchain.callbacks import get_openai_callback

Setting the default environment API key for the OpenAI API python client.

In [ ]:
# openai.api_key = os.environ["OPENAI_API_KEY"]

List of all OpenAI model listing the root model and release date, sorted by release date with the most recent at the top.

In [ ]:

    # *,
    # verbose: bool = None,
    # callback_manager: langchain.callbacks.base.BaseCallbackManager = None,
    # client = None,
    # model_name: str = 'gpt-3.5-turbo',
    # temperature: float = 0.7,
    # model_kwargs: Dict[str, Any] = None,
    # openai_api_key: Optional[str] = None,
    # request_timeout: int = 60,
    # max_retries: int = 6,
    # streaming: bool = False,
    # n: int = 1,
    # max_tokens: Optional[int] = None,

In [ ]:
from typing import Any, Dict, List, Union
from langchain.schema import AgentAction, AgentFinish, LLMResult
import queue
import threading

## **Streaming Callback**

For some reason on_llm_new_token is the only callback function that actually gets called.

In [ ]:
class StreamingCallbackHandler(BaseCallbackHandler):
    def __init__(self, token_callback_func=None, stream=False):
        super().__init__()
        self.token_callback_func = token_callback_func
        self.stream = stream
        
    def on_llm_new_token(self, token: str, **kwargs):
        """Run on new LLM token. Only available when streaming is enabled."""
        print('NEW TOKEN', kwargs)
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(token)
        
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs
    ) -> None:
        """Run when LLM starts running."""

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        """Run when LLM ends running."""
        print('on_llm_end called')
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(None)

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs
    ) -> None:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        """Run when chain starts running."""

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        """Run when chain ends running."""
        print('on_chain_end called')
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(None)

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs
    ) -> None:
        """Run when chain errors."""

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs
    ) -> None:
        """Run when tool starts running."""

    def on_agent_action(self, action: AgentAction, **kwargs) -> Any:
        """Run on agent action."""
        pass

    def on_tool_end(self, output: str, **kwargs) -> None:
        """Run when tool ends running."""
        print('on_tool_end called')
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(None)


    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs
    ) -> None:
        """Run when tool errors."""

    def on_text(self, text: str, **kwargs) -> None:
        """Run on arbitrary text."""
        print('on_text called')
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(None)

    def on_agent_finish(self, finish: AgentFinish, **kwargs) -> None:
        """Run on agent end."""
        print('on_agent_finish called')
        if self.stream and self.token_callback_func is not None:
            self.token_callback_func(None)


In [ ]:
chat = ChatOpenAI(model_name = 'gpt-3.5-turbo', streaming=True, 
                  callback_manager=CallbackManager(
                      [StreamingCallbackHandler(token_callback_func=lambda o: print(o,end=''),stream=True)]
                  ), verbose=True, temperature=0)
resp = chat([HumanMessage(content="Write me a short haiku about sparkling water.")])


NEW TOKEN {}
NEW TOKEN {}
BNEW TOKEN {}
ubblesNEW TOKEN {}
 danceNEW TOKEN {}
 andNEW TOKEN {}
 shineNEW TOKEN {}
,
NEW TOKEN {}
SparkNEW TOKEN {}
lingNEW TOKEN {}
 waterNEW TOKEN {}
 onNEW TOKEN {}
 myNEW TOKEN {}
 tongueNEW TOKEN {}
,
NEW TOKEN {}
RefreshingNEW TOKEN {}
 andNEW TOKEN {}
 pureNEW TOKEN {}
.NEW TOKEN {}


In [ ]:
resp

AIMessage(content='Bubbles dance and shine,\nSparkling water on my tongue,\nRefreshing and pure.', additional_kwargs={})

## Callback To Generator Wrapper

In [ ]:
class CallbackToGeneratorConvertor:
    def __init__(self):
        self.q = queue.Queue()
        self.generator_active = True

    def callback(self, data):
        self.q.put(data)
        if data is None:
            self.q.put(None)

    def generator(self):
        self.generator_active = True
        while self.generator_active:
            try:
                data = self.q.get(True,0.01)
                if data is None:
                    self.generator_active = False
                    break
                else:
                    yield data
            except queue.Empty:
                pass
                # print('queue empty')

In [ ]:
callback_to_generator_convertor = CallbackToGeneratorConvertor()

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-3.5-turbo', streaming=True, 
                  callback_manager=CallbackManager([StreamingCallbackHandler(token_callback_func=callback_to_generator_convertor.callback,
                                                                             stream=True)]), verbose=True, temperature=0)


In [ ]:
chat([HumanMessage(content="Write me a short haiku about sparkling water.")])

NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}


AIMessage(content='Bubbles dance and shine,\nSparkling water on my tongue,\nRefreshing and pure.', additional_kwargs={})

In [ ]:
chat.model_name = 'gpt-3.5-turbo'

# chat([HumanMessage(content="Write me a short haiku about sparkling water.")])
thread = threading.Thread(target=chat, args=([HumanMessage(content="Write me a short haiku about sparkling water.")],))
thread.start()

# [print('x',o) for o in callback_to_generator_convertor.generator()]

In [ ]:
callback_to_generator_convertor.q.get(False)

''

In [ ]:
thread.is_alive()

True

## Gradio Interface

Will add this back once streaming gets fixed.

TODO: 
- Add tiktoken token counter for streaming api. https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
- Add running total calculator to track tokens and $.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
NEW TOKEN {}
